# Token API

In [ ]:
#| default_exp token

In [ ]:
#| export

from typing import Tuple
from dataclasses import dataclass
from sugar.helpers import normalize_address, float_to_uint256

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

@dataclass(frozen=True)
class Token:
    """Data class for Token
    based on: https://github.com/velodrome-finance/sugar/blob/v2/contracts/LpSugar.vy#L17
    """

    chain_id: str
    chain_name: str
    token_address: str
    symbol: str
    decimals: int
    listed: bool
    wrapped_token_address: str = None

    def __eq__(self, other):
        t1, t2 = normalize_address(self.wrapped_token_address or self.token_address), normalize_address(other.wrapped_token_address or other.token_address)
        return  t1 == t2 and self.chain_id == other.chain_id

    def parse_units(self, value: float) -> int:
        """Convert a value to wei/kwei/gwei/mwei based on the token's decimals."""
        return float_to_uint256(value=value, decimals=self.decimals)

    def to_float(self, value: int) -> float:
        """Convert a value from wei/kwei/gwei/mwei to decimal based on the token's decimals."""
        return float(value / 10 ** self.decimals)

    @property
    def is_native(self) -> bool: return self.wrapped_token_address is not None

    @classmethod
    def make_native_token(cls, symbol: str, wrapped_address: str, decimals: int, chain_id: str, chain_name: str) -> "Token":
        return Token(chain_id=chain_id, chain_name=chain_name, token_address=symbol, symbol=symbol, decimals=decimals, listed=True, wrapped_token_address=wrapped_address)

    @classmethod
    def from_tuple(cls, t: Tuple, chain_id: str, chain_name: str) -> "Token":
        (token_address, symbol, decimals, _, listed) = t
        return Token(
            chain_id=chain_id,
            chain_name=chain_name,
            token_address=normalize_address(token_address),
            symbol=symbol,
            decimals=decimals,
            listed=listed,
        )

In [ ]:
usdc = Token(chain_id="10", chain_name="OP", token_address='0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85', symbol='USDC', decimals=6, listed=True, wrapped_token_address=None)
velo = Token(chain_id="10", chain_name="OP", token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', symbol='VELO', decimals=18, listed=True, wrapped_token_address=None)
another_velo = Token(chain_id="10", chain_name="OP", token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db'.lower(), symbol='VELO', decimals=18, listed=True, wrapped_token_address=None)
velo_poser = Token(chain_id="101", chain_name="OOPS", token_address='0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', symbol='VELO', decimals=18, listed=True, wrapped_token_address=None)
eth = Token(chain_id="10", chain_name="OP", token_address='ETH', symbol='ETH', decimals=18, listed=True, wrapped_token_address='0x4200000000000000000000000000000000000006') 

assert usdc != velo, "USDC and VELO should not be equal"
assert velo == another_velo, "VELO and another VELO should be equal"
assert velo == velo, "VELO and VELO should be equal"
assert velo != velo_poser, "VELO and VELO poser should not be equal"
assert velo != eth, "VELO and ETH should not be equal"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()